In [4]:
pip install -qU langchain-community

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install -qU langchain-community faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install -qU langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install pypdf

  Using cached pypdf-5.1.0-py3-none-any.whl.metadata (7.2 kB)
Using cached pypdf-5.1.0-py3-none-any.whl (297 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

In [20]:
path = "./data/understandingclimatechange.pdf"

In [23]:
def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)

    embeddings = OllamaEmbeddings(
        model="llama3.2:3b-instruct-fp16",
    )

    # Create vector store
    vectorstore = FAISS.from_documents(texts, embeddings)

    return vectorstore

In [24]:
chunks_vector_store = encode_pdf(path, chunk_size=1000, chunk_overlap=200)

In [25]:
chunks_query_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 2})

In [30]:
test_query = "What is the main cause of climate change?"
context = chunks_query_retriever.invoke(test_query)
for i, doc in enumerate(context):
    print(f'Context #{i+1}')
    print(doc.page_content)

Context #1
atmospheric carbon dioxide has increased by more than 30 percent during the same period due to 
human activity, predominantly the burning of fossil fuels and to a lesser extent deforestation and 
other land use changes, along with industrial activities such as cement production.
Context #2
now the issue of climate change has  been confined almost entirely to the research community 
and very difficult concept for many people to understand  considering the rise of climate 
skepticism (Crutezen, 2000, Steffen, 2002). 
 
What is Climate? 
The term ‗climate‘  refers to a statistical descr iption of weather in terms of the mean and 
variability of temperature, precipitation and wind over a period of time, ranging from months to 
millions of years (the classical period is 30 - 35 years) and of the related conditions of oceans , 
land surfaces including human perturbations and ice sheets(Hoyt and Schatter, 1997).The climate 
in any given area is actuallya complex and interactive sys